# Nevral stiloverføring

## Demo

Install pystiche:

In [ ]:
!pip install pystiche

Følgende demo er hentet fra: https://docs.pystiche.org/en/latest/galleries/examples/beginner/example_nst_with_pystiche.html#sphx-glr-download-galleries-examples-beginner-example-nst-with-pystiche-py

In [1]:
import pystiche

from pystiche import demo, enc, loss, optim
from pystiche.image import show_image, read_image, write_image
from pystiche.misc import get_device, get_input_image

device = get_device()
print(f"Device: {device}")

Device: cpu


Setter opp multi-layer encoder

In [2]:
multi_layer_encoder = enc.vgg19_multi_layer_encoder()

Laster ned demo-bilder.

In [3]:
images = demo.images()
images.download()

Lager en funksjon NST(stilbilde, innholdsbilde).perceptual_loss = loss.PerceptualLoss(content_loss, style_loss).to(device)

In [4]:
def NST(content_image, 
        style_image,
        random_input = False,
        steps = 500,
        content_weight = 1e0, 
        style_weight = 1e3,
        content_layer = "relu4_2",
        style_layers = ("relu1_1", "relu2_1", "relu3_1", "relu4_1", "relu5_1")):
    
    # Create content loss function
    content_encoder = multi_layer_encoder.extract_encoder(content_layer)
    
    content_loss = loss.FeatureReconstructionLoss(content_encoder, 
                                                  score_weight=content_weight)
    
    # Create style loss function
    def get_style_op(encoder, layer_weight):
        return loss.GramLoss(encoder, score_weight=layer_weight)

    style_loss = loss.MultiLayerEncodingLoss(
         multi_layer_encoder, style_layers, get_style_op, score_weight=style_weight)
    
    # Create combined loss function
    perceptual_loss = loss.PerceptualLoss(content_loss, style_loss).to(device)
    
    
    perceptual_loss.set_content_image(content_image)
    perceptual_loss.set_style_image(style_image)
    
    # Set input image
    if random_input:
        starting_point = "random"
    else:
        starting_point = "content"
    
    input_image = get_input_image(starting_point, content_image=content_image)
    
    # Generate NST output image
    output_image = optim.image_optimization(input_image, perceptual_loss, num_steps=steps)
    return output_image

def demo_NST(content_name, style_name):
    size = 500
    content_image = images[content_name].read(size=size, device=device)
    style_image = images[style_name].read(size=size, device=device)
    return NST(content_image, style_image)

## Testing av NST

Først skriver vi ut alle demobildene vi bruker:

In [ ]:
size = 500
img_names = ["bird1", "paint", "bird2", "mosaic", "castle", "church", "cliff"]
for name in img_names:
    show_image(images[name].read(size=size, device=device))

### Bilder med standardkonfigurasjon

Vi prøver forskjellige bilder med standardkonfigurasjonen:

In [ ]:
show_image(demo_NST("bird1", "castle"))

In [ ]:
show_image(demo_NST("bird2", "paint"))

In [ ]:
show_image(demo_NST("cliff", "mosaic"))

In [ ]:
show_image(demo_NST("castle", "church"))

### Bilder i forskjellige størrelser

Vi bruker standardkonfigurasjonen, bortsett fra at vi bruker 4 forskjellige bildestørrelser: 100, 300, 500, 800. Det vil si at antall steg er konstant for alle 4 bildene. Vi bruker bird1 som innhold og paint som stilbilde.

In [ ]:
sizes = (100, 300, 500, 800)
for size in sizes:
    content_image = images["bird1"].read(size=size, device=device)
    style_image = images["paint"].read(size=size, device=device)
    show_image(NST(content_image, style_image))
    

Bildene blir mer detaljerte med høyere oppløsning, som gir mening.

### Bilder med forskjellig antall steg

Vi bruker NST med samme bilder og standardoppsettet, bortsett fra at vi endrer antall steg: vi prøver 31, 62, 125, 250, 500, 1000 steg.

In [ ]:
size = 500
content_image = images["bird1"].read(size=size, device=device)
style_image = images["paint"].read(size=size, device=device)

step_list = (31, 62, 125, 250, 500, 1000)

for steps in step_list:
    show_image(NST(content_image, style_image, steps=steps))

Vi ser at bildet gradvis får en stil som likner bildet paint, desto flere steg algoritmen tar. I starten ligner imidlertid bildet veldig på innholdsbildet. Dette gir mening, da vi starter med innholdsbilde, og gradvis endrer bildet slik at det får stil liknende stilbildet. Vi merker oss at det både tar lengre tid, og at loss-funksjonen får til å få lavere verdi, når vi øker antall steg.

### Forskjellige vektinger av innhold og stil

Vi bruker de samme bildene, men velger denne gangen å variere vektingen mellom innhold og stil. Vi holder innholdsvektingen konstant på 1, men prøver med stilvekting på 1e0, 1e1, 1e2, 1e3:

In [ ]:
size = 500
content_image = images["bird1"].read(size=size, device=device)
style_image = images["paint"].read(size=size, device=device)

style_weights = (1e0, 1e2, 1e4, 1e6)

for style_weight in style_weights:
    show_image(NST(content_image, style_image, style_weight=style_weight))

## Hva hvis vi fjerner innholdsbildet?

Nå ønsker vi å se på effekten av å sette innholdsvektingen til null og starte med tilfeldig støy som input.

In [ ]:
size = 500
stylenames = ["paint", "church", "mosaic", "bird1"]
for stylename in stylenames:
    content_image = images["bird1"].read(size=size, device=device)
    style_image = images[stylename].read(size=size, device=device)
    show_image(NST(content_image, style_image, content_weight=0, random_input = True))

We repeat the same with fewer style layers:

In [ ]:
style_layers = ("relu1_1", "relu2_1")
for stylename in stylenames:
    content_image = images["bird1"].read(size=size, device=device)
    style_image = images[stylename].read(size=size, device=device)
    show_image(NST(content_image, style_image, content_weight=0, random_input = True, 
                   style_layers=style_layers))

In [ ]:
style_layers = ("relu1_1",)
for stylename in stylenames:
    content_image = images["bird1"].read(size=size, device=device)
    style_image = images[stylename].read(size=size, device=device)
    show_image(NST(content_image, style_image, content_weight=0, random_input = True, 
                   style_layers=style_layers))

Vi ser færre detaljer i bildene som produseres med færre lag. 

TODO: Forklaring. Ideen er noe som at flere lag til sammen kan skape mer avanserte, distinkte mønstre, flere filtre oppå hverandre kan inneholde mye informasjon. Vi ser bl.a. noe som ser ut som nebb og øyne fra bird1 på bildet med 4 lag, mens det er mye vanskeligere å gjenkjenne deler av fuglen på bildet med 1 eller 2 stillag.

## Nettbutikken Frida feirer jul

Nettbutikken Frida som selger dagligvarer feirer jul, og ønsker å tilpasse bildene av varene deres til julesesongen. Derfor ønsker de å ta i bruk nevral stiloverføring for å effektivt lage bilder med julestil.

### Bilder med julestil

Først må vi finne noen gode bilder med julestil. Vi har lastet inn bildene som et privat datasett med kaggle. Vi har også lagt ved bildene i data-mappen. Det første bildet er hentet fra: https://www.flickr.com/photos/30478819@N08/51393913903 (Marco Verch, 2021). 

In [5]:
size = 500

path = "../input/christmas-pictures/christmas_pictures/merry_christmas.jpg"
christmas1 = read_image(path, size=size, device=device)
show_image(christmas1)

FileNotFoundError: [Errno 2] No such file or directory: '../input/christmas-pictures/christmas_pictures/merry_christmas.jpg'

Det andre bildet er hentet fra: https://commons.wikimedia.org/wiki/File:Christmas_Tree_and_Presents.jpg (alhill42, 2014). 

In [ ]:
path = "../input/christmas-pictures/christmas_pictures/Christmas_Tree_and_Presents.jpg"
christmas2 = read_image(path, size=size, device=device)
show_image(christmas2)

Det tredje bildet er hentet fra: https://pixabay.com/illustrations/christmas-winter-sleigh-santa-claus-6756684/ (Darkmoon art, 2021). 

In [ ]:
path = "../input/christmas-pictures/christmas_pictures/snow_and_santa.jpg"
christmas3 = read_image(path, size=size, device=device)
show_image(christmas3)

### Bilder av Fridas varer

Vi har kjørt denne notebooken med kaggle, og har lastet inn bildemappen som et datasett. Lenke til datasettet kan finnes her: https://www.kaggle.com/datasets/eb3b62569c5ad63905362b110520aa28e0562e71b65a6db1942ffdd06ed7bbdf?select=flowers.jpeg. Vi har imidlertid måttet modifisere bildene litt: vi har fjernet alfa-kanalen manuelt fra enkelte bilder med bilderedigeringsprogrammet GIMP slik at koden skulle kunne kjøre. Vi legger bildene vi har brukt inn i mappen data.

In [ ]:
import os

# Store the image paths for each image in dictionary
image_paths = {}

directory = "../input/hon2200-frida/hon2200_frida_bilder"
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    name = filename.split(".")[0]
    image_paths[name] = path
    
print(image_paths.keys())

In [ ]:
size = 500
for name in image_paths:
    img = read_image(image_paths[name], size=size, device=device)
    show_image(img)

### Fridas varer med julestil

In [ ]:
# code for testing, to be removed
"""
size = 500
style_image = christmas2
content_image = read_image(image_paths["cookie"], size=size, device=device)

print(content_image.shape)
print(style_image.shape)

#content_image = content_image[:,:3,:,:]
#translucent_mask = cont
#show_image(content_image)
show_image(NST(content_image, style_image))
"""

In [ ]:
size = 500
style_image = christmas1
for key in image_paths:
    content_image = read_image(image_paths[key], size=size, device=device)
    output = NST(content_image, style_image)
    write_image(output, key + "1.jpeg")
    show_image(output)

Todo

In [ ]:
size = 500
style_image = christmas2
for key in image_paths:
    content_image = read_image(image_paths[key], size=size, device=device)
    output = NST(content_image, style_image)
    write_image(output, key + "2.jpeg")
    show_image(output)

In [ ]:
style_image = christmas3
for key in image_paths:
    content_image = read_image(image_paths[key], size=size, device=device)
    output = NST(content_image, style_image)
    write_image(output, key + "3.jpeg")
    show_image(output)